In [5]:
'''
import pandas as pd

labels = pd.read_csv("F:/AI Project/churn-retention-platform/data/raw/train.csv")
#labels.head()
#labels.shape
#labels['is_churn'].value_counts()
#labels['is_churn'].value_counts(normalize=True)
labels['msno'].is_unique
'''




'\nimport pandas as pd\n\nlabels = pd.read_csv("F:/AI Project/churn-retention-platform/data/raw/train.csv")\n#labels.head()\n#labels.shape\n#labels[\'is_churn\'].value_counts()\n#labels[\'is_churn\'].value_counts(normalize=True)\nlabels[\'msno\'].is_unique\n'

In [6]:
import pandas as pd

# Read ONLY transaction_date to get max date
date_col = pd.read_csv(
    "F:/AI Project/churn-retention-platform/data/raw/transactions.csv",
    usecols=["transaction_date"],
    dtype={"transaction_date": "int32"}
)

latest_date = pd.to_datetime(
    date_col["transaction_date"].max(),
    format="%Y%m%d"
)

cutoff_30 = latest_date - pd.Timedelta(days=30)

del date_col  # free memory


In [7]:
print(cutoff_30)

2017-01-29 00:00:00


In [1]:
import pandas as pd
from collections import defaultdict

labels = pd.read_csv("F:/AI Project/churn-retention-platform/data/raw/train.csv")

# Read ONLY transaction_date to get max date
date_col = pd.read_csv(
    "F:/AI Project/churn-retention-platform/data/raw/transactions.csv",
    usecols=["transaction_date"],
    dtype={"transaction_date": "int32"}
)

latest_date = pd.to_datetime(
    date_col["transaction_date"].max(),
    format="%Y%m%d"
)

cutoff_30 = latest_date - pd.Timedelta(days=30)

del date_col  # free memory

agg = defaultdict(lambda: {
    "total_amount_paid": 0.0,
    "total_plan_days": 0,
    "num_transactions": 0,
    "num_cancellations": 0
})

chunk_iter = pd.read_csv(
    "../data/raw/transactions.csv",
    usecols=[
        "msno",
        "transaction_date",
        "actual_amount_paid",
        "payment_plan_days",
        "is_cancel"
    ],
    dtype={
        "msno": "category",
        "transaction_date": "int32",
        "actual_amount_paid": "float32",
        "payment_plan_days": "int16",
        "is_cancel": "int8"
    },
    chunksize=500_000
)

for chunk in chunk_iter:

    # Convert date only for the chunk
    chunk["transaction_date"] = pd.to_datetime(
        chunk["transaction_date"],
        format="%Y%m%d"
    )

    # Filter to 30-day window early
    chunk = chunk[chunk["transaction_date"] >= cutoff_30]


chunk.describe()






,payment_plan_days,actual_amount_paid,transaction_date,is_cancel
count,2369.000000,2369.000000,2369,2369.000000
mean,30.018995,129.336014,2017-02-12 20:55:12.790206976,0.032081
min,30.000000,0.000000,2017-01-29 00:00:00,0.000000
25%,30.000000,99.000000,2017-02-03 00:00:00,0.000000
50%,30.000000,149.000000,2017-02-12 00:00:00,0.000000
75%,30.000000,149.000000,2017-02-22 00:00:00,0.000000
max,60.000000,180.000000,2017-02-28 00:00:00,1.000000
std,0.689002,29.573406,NaN,0.176253


In [ ]:
import pandas as pd
from collections import defaultdict

labels = pd.read_csv("F:/AI Project/churn-retention-platform/data/raw/train.csv")

# Read ONLY transaction_date to get max date
date_col = pd.read_csv(
    "F:/AI Project/churn-retention-platform/data/raw/transactions.csv",
    usecols=["transaction_date"],
    dtype={"transaction_date": "int32"}
)

latest_date = pd.to_datetime(
    date_col["transaction_date"].max(),
    format="%Y%m%d"
)

cutoff_30 = latest_date - pd.Timedelta(days=30)

del date_col  # free memory

agg = defaultdict(lambda: {
    "total_amount_paid": 0.0,
    "total_plan_days": 0,
    "num_transactions": 0,
    "num_cancellations": 0
})

chunk_iter = pd.read_csv(
    "../data/raw/transactions.csv",
    usecols=[
        "msno",
        "transaction_date",
        "actual_amount_paid",
        "payment_plan_days",
        "is_cancel"
    ],
    dtype={
        "msno": "category",
        "transaction_date": "int32",
        "actual_amount_paid": "float32",
        "payment_plan_days": "int16",
        "is_cancel": "int8"
    },
    chunksize=500_000
)

for chunk in chunk_iter:

    # Convert date only for the chunk
    chunk["transaction_date"] = pd.to_datetime(
        chunk["transaction_date"],
        format="%Y%m%d"
    )

    # Filter to 30-day window early
    chunk = chunk[chunk["transaction_date"] >= cutoff_30]

    # Aggregate row by row (fast enough at chunk scale)
    for row in chunk.itertuples(index=False):
        user = row.msno

        agg[user]["total_amount_paid"] += row.actual_amount_paid
        agg[user]["total_plan_days"] += row.payment_plan_days
        agg[user]["num_transactions"] += 1
        agg[user]["num_cancellations"] += row.is_cancel

    del chunk  # free memory
    

features_tx_30 = (
    pd.DataFrame.from_dict(agg, orient="index")
      .reset_index()
      .rename(columns={"index": "msno"})
)

dataset_v1 = labels.merge(
    features_tx_30,
    on="msno",
    how="left"
)

dataset_v1.fillna(0, inplace=True)

'''
dataset_v1.groupby("is_churn")[
    ["total_amount_paid", "num_transactions", "num_cancellations"]
].mean()
'''

dataset_v1.to_csv(
    "../data/processed/dataset_v1.csv",
    index=False
)





